# Instal·lar importar llibreries necessaries

In [40]:
!pip install pyagrum

In [41]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [42]:
gum.config['notebook','potential_visible_digits'] = 2

In [43]:
def testIndep(bn,x,y,coneixem):
    resultat="" if bn.isIndependent(x,y,coneixem) else "NO "
    donades="." if len(coneixem)==0 else f" donats {coneixem}."
    print(f"{x} i {y} {resultat}són independents{donades}")

# Constriuir l'estructura de la xarxa

In [44]:
bn=gum.BayesNet("Problemes respiratioris")

In [45]:
# Afegir tots els nodes
bn.add(gum.LabelizedVariable('Visita Àsia'))
bn.add(gum.LabelizedVariable('Dispnea'))
bn.add(gum.LabelizedVariable('Tuberculosi o Càncer'))
bn.add(gum.LabelizedVariable('Bronquitis'))
bn.add(gum.LabelizedVariable('Fumador'))
bn.add(gum.LabelizedVariable('Càncer de Pulmó'))
bn.add(gum.LabelizedVariable('Raigs X positius'))
bn.add(gum.LabelizedVariable('Tuberculosi'))

7

In [46]:
# Afegir conexions
bn.addArc("Visita Àsia","Tuberculosi")
bn.addArc("Tuberculosi", "Tuberculosi o Càncer")
bn.addArc("Tuberculosi o Càncer","Raigs X positius")
bn.addArc("Bronquitis","Dispnea")
bn.addArc("Tuberculosi o Càncer","Dispnea")
bn.addArc("Fumador","Càncer de Pulmó")
bn.addArc("Fumador","Bronquitis")
bn.addArc("Càncer de Pulmó","Tuberculosi o Càncer")

In [47]:
bn

(pyAgrum.BayesNet<double>@0x5b0a973923a0) BN{nodes: 8, arcs: 8, domainSize: 256, dim: 18, mem: 288o}

# Definim probabilitats

In [48]:
bn.cpt("Fumador").fillWith([0.5,0.5])
bn.cpt("Visita Àsia").fillWith([0.99,0.01])
bn.cpt("Càncer de Pulmó")[{'Fumador':0}]=[0.99,0.01]
bn.cpt("Càncer de Pulmó")[{'Fumador':1}]=[0.90,0.10]

bn.cpt("Bronquitis")[{'Fumador':0}]=[0.70,0.30]
bn.cpt("Bronquitis")[{'Fumador':1}]=[0.40,0.60]

In [49]:
bn.cpt("Tuberculosi")[{'Visita Àsia':0}]=[0.99,0.01]
bn.cpt("Tuberculosi")[{'Visita Àsia':1}]=[0.95,0.05]

In [50]:
bn.cpt("Dispnea")[{'Tuberculosi o Càncer':0,'Bronquitis':0}]=[0.90,0.10]
bn.cpt("Dispnea")[{'Tuberculosi o Càncer':0,'Bronquitis':1}]=[0.20,0.80]
bn.cpt("Dispnea")[{'Tuberculosi o Càncer':1,'Bronquitis':0}]=[0.30,0.70]
bn.cpt("Dispnea")[{'Tuberculosi o Càncer':1,'Bronquitis':1}]=[0.10,0.90]

In [51]:
bn.cpt("Raigs X positius")[{'Tuberculosi o Càncer':0}]=[0.95,0.05]
bn.cpt("Raigs X positius")[{'Tuberculosi o Càncer':1}]=[0.02,0.98]

In [52]:
bn.cpt("Tuberculosi o Càncer")[{'Càncer de Pulmó':0, 'Tuberculosi':0}]=[1,0]
bn.cpt("Tuberculosi o Càncer")[{'Càncer de Pulmó':1, 'Tuberculosi':0}]=[0,1]
bn.cpt("Tuberculosi o Càncer")[{'Càncer de Pulmó':0, 'Tuberculosi':1}]=[0,1]
bn.cpt("Tuberculosi o Càncer")[{'Càncer de Pulmó':1, 'Tuberculosi':1}]=[0,1]

In [53]:
gnb.sideBySide(bn.cpt("Fumador"),bn.cpt("Càncer de Pulmó"),bn.cpt("Visita Àsia"),captions=['P(Fumador)','P(Càncer de Pulmó)','P(Visita Àsia)'])
gnb.sideBySide(bn.cpt("Bronquitis"),gnb.getBN(bn),bn.cpt("Tuberculosi"), captions=['P(Bronquitis|Fumador)','','P(Tuberculosi|Visita Àsia)'])
gnb.sideBySide(bn.cpt("Dispnea"),bn.cpt("Raigs X positius"),bn.cpt("Tuberculosi o Càncer"),
  captions=['P(Dispnea|Bronquitis,Tuberculosi o Càncer)','P(Raigs X positius|Tuberculosi o Càncer)','P(Tuberculosi o Càncer|Tuberculosi,Càncer de Pulmó)'])

# 1. Donada la informació que un pacient té bronquitis, quina és la probabilitat que tingui tuberculosi sense saber si ha visitat Àsia?

In [58]:
ie=gum.LazyPropagation(bn)
ie.setEvidence({'Bronquitis':1})
ie.makeInference()
print("%.0f%%" % (ie.posterior("Tuberculosi")[1]*100))

1%


La probabilitat de que el pacient tingui tuberculosi sense sapiguer si ha visitat asia i sapiguent que té bronquitis es d'un 1%


# 2. Si sabem que un pacient té tuberculosi i NO fuma, quina és la probabilitat que tingui dispnea?

In [57]:
ie=gum.LazyPropagation(bn)
ie.setEvidence({'Tuberculosi':1,'Fumador':0})
ie.makeInference()
print("%.0f%%" % (ie.posterior("Dispnea")[1]*100))

76%


La probabilitat de que un pacient tingui Dispnea sapiguent que te tuberculosi i que no fuma es d'un 76%

# 3. Quina és la probabilitat que un pacient tingui bronquitis si sabem que té dispnea, però no tenim informació sobre si ha tingut contacte amb persones amb tuberculosi?

In [15]:
ie=gum.LazyPropagation(bn)
ie.setEvidence({'Dispnea':1})
ie.makeInference()
print("%.0f%%" % (ie.posterior("Bronquitis")[1]*100))

83%


La probabilitat de que un pacient tingui Bronquits sense sapiguer si ha tingut contacte amb algú amb tuberculosi, però sabem que te Dispnea es d'un 83%

# 4. Si un pacient té tuberculosi, quina és la probabilitat que NO hagi visitat Àsia en % sense decimals?

In [20]:
ie=gum.LazyPropagation(bn)
ie.setEvidence({'Tuberculosi':1})
ie.makeInference()
print("%.0f%%" % (ie.posterior("Visita Àsia")[0]*100))

95%


Un pacient que amb tuberculosi té un 95% de probabilitats de no haber visitat Àsia

# 5. Són independents els nodes bronquitis i dispnea donats els nodes tuberculosi i fumador?

In [33]:
testIndep(bn,"Bronquitis","Dispnea",['Tuberculosi','Fumador'])

Bronquitis i Dispnea NO són independents donats ['Tuberculosi', 'Fumador'].


# 6. Són independents els nodes bronquitis i tuberculosi donats els nodes fumador i 'Visita Àsia'?

In [34]:
testIndep(bn,"Bronquitis","Tuberculosi",['Fumador','Visita Àsia'])

Bronquitis i Tuberculosi són independents donats ['Fumador', 'Visita Àsia'].


# 7. És independent tenir tuberculosi de tenir càncer de pulmó?

In [35]:
testIndep(bn,"Tuberculosi","Càncer de Pulmó",[])

Tuberculosi i Càncer de Pulmó són independents.


# 8. Quina és la probabilitat de tenir càncer de pulmó?

In [36]:
ie=gum.LazyPropagation(bn)
ie.makeInference()
print("%.0f%%" % (ie.posterior("Càncer de Pulmó")[1]*100))

6%


Hi ha un 6% de probabilitats de tenir càncer de pulmó sense conèixer cap altre factor

# 9. Quina és la probabilitat de tenir tuberculosi?

In [37]:
ie=gum.LazyPropagation(bn)
ie.makeInference()
print("%.0f%%" % (ie.posterior("Tuberculosi")[1]*100))

1%


Hi ha un 1% de probabilitats de tenir tuberculosi sense conèixer cap altre factor

# 10. Quina és la probabilitat de tenir tuberculosi o càncer?

In [39]:
ie=gum.LazyPropagation(bn)
ie.makeInference()
print("%.0f%%" % (ie.posterior("Tuberculosi o Càncer")[1]*100))

6%


Hi ha un 6% de probabilitats de tenir Tuberculosi o Càncer sense conèixer cap altre factor